In [1]:
!date

Tue Apr 12 14:06:58 EDT 2022


In [2]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [3]:
%%time

import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from helper_funcs import *
import shutil
from scipy.stats import ttest_ind,ttest_1samp,ttest_rel

import umap


from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

CPU times: user 13.4 s, sys: 1.9 s, total: 15.3 s
Wall time: 26.1 s


In [4]:
def dummy_ordinal(invec):
    unique_values = np.unique(invec)
    new_values = np.arange(1,len(unique_values)+1)
    new_arr = [new_values[val==unique_values][0] for val in invec]
    return new_arr

In [5]:
df = pd.read_csv('../Data/comb_df.csv')
df['dataset_id'] = dummy_ordinal(df['dataset'])
df['site_id'] = dummy_ordinal(df['site'])


u_sites = np.unique(df['site_id'].values)
sites = df['site_id'].values.astype(float)
site_ratios = np.array([(df['diag'].values[df['site_id'].values==s]==1).mean() for s in u_sites])
bad_sites = u_sites[abs(site_ratios-.5)>.1]
sites[df['site_id'].isin(bad_sites).values] = np.nan
df['sites_bal'] = sites
print((~np.isnan(df['sites_bal'].values)).sum())


patients = df['diag'].values==1
controls = df['diag'].values==2

df_asd = df.iloc[patients]
df_td = df.iloc[~patients]

print(df_asd.shape)
print(df_td.shape)

df

1025
(661, 16)
(841, 16)


,Unnamed: 0,participant_id,diag,age,sex,fiq,site,DSMIV,ados_total,ados_social,ados_comm,ados_rrb,dataset,dataset_id,site_id,sites_bal
0,0,50002,1,16.77,1,103.0,13,1.0,12.0,8.0,4.0,3.0,ABIDE I,1,4,4.0
1,2,50004,1,19.09,1,113.0,13,1.0,18.0,12.0,6.0,2.0,ABIDE I,1,4,4.0
2,3,50005,1,13.73,2,119.0,13,1.0,12.0,8.0,4.0,1.0,ABIDE I,1,4,4.0
3,4,50006,1,13.37,1,109.0,13,1.0,12.0,8.0,4.0,4.0,ABIDE I,1,4,4.0
4,9,50011,1,16.93,1,111.0,13,1.0,13.0,9.0,4.0,NaN,ABIDE I,1,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,746,30163,2,8.00,2,136.0,ABIDEII-OHSU_1,NaN,NaN,NaN,NaN,NaN,ABIDE II,2,29,NaN
1498,747,30164,2,10.00,2,115.0,ABIDEII-OHSU_1,NaN,NaN,NaN,NaN,NaN,ABIDE II,2,29,NaN
1499,748,30165,2,12.00,2,120.0,ABIDEII-OHSU_1,NaN,NaN,NaN,NaN,NaN,ABIDE II,2,29,NaN
1500,749,30166,2,10.00,2,112.0,ABIDEII-OHSU_1,NaN,NaN,NaN,NaN,NaN,ABIDE II,2,29,NaN


In [6]:
def flatten_cmat(cmats):
    ns = cmats.shape[0]
    ni = cmats.shape[-1]
    tridx = np.triu_indices(n=ni,k=1)
    mat_flat = np.array([cmats[i,:,:][tridx] for i in range(ns)])
    return mat_flat

In [7]:
def depad(mat,idx=(6,57)):
    'depads the cmats'
    if mat.ndim==3:
        mat_trimmed = mat[:,idx[0]:idx[1],idx[0]:idx[1]]
    elif mat.ndim==4:
        mat_trimmed = mat[:,:,idx[0]:idx[1],idx[0]:idx[1]]
    else:
        print(mat.shape)
        raise Exception("Not implemented")
        
    return mat_trimmed

In [8]:
## Load Data
cmats = np.load('../Data/cmats_r51_S1502.npz')['data']
cmats_rel = np.load('../Data/rel-cmats_r51_S1502.npz')['data']
cmats_asd_flat = flatten_cmat(cmats[patients,:,:])
cmats_td_flat = flatten_cmat(cmats[~patients,:,:])

print(cmats.shape)
print(cmats_rel.shape)
print(cmats_asd_flat.shape)

(1502, 51, 51)
(1502, 2, 51, 51)
(661, 1275)


In [9]:
%%time

analysis_name = 'CVAE_2022-03-25 18:28:49.469238'
save_dir = os.path.join('../Assets/tf_weights',analysis_name)

data = np.load(os.path.join(save_dir,'results.npz'))
data = dict(data)
data_keys = list(data.keys())

data['recon_td_mu'] = depad(data['recon_td_mu'])
data['recon_asd_mu'] = depad(data['recon_asd_mu'])
data['recon_twin_mu'] = depad(data['recon_twin_mu'])
data['recon_td_samples'] = depad(data['recon_td_samples'])
data['recon_asd_samples'] = depad(data['recon_asd_samples'])
data['recon_twin_samples'] = depad(data['recon_twin_samples'])

data['Z_asd_sample10'] = data['Z_sample100'][0:10,patients,:]
data['S_asd_sample10'] = data['S_sample100'][0:10,patients,:]

for key in data_keys:
    print(f'{key.center(20)} | {data[key].shape}')

        Z_mu         | (1502, 16)
      Z_sigma        | (1502, 16)
         Z           | (1502, 16)
        S_mu         | (1502, 16)
      S_sigma        | (1502, 16)
         S           | (1502, 16)
    Z_sample100      | (100, 1502, 16)
    S_sample100      | (100, 1502, 16)
    recon_td_mu      | (841, 51, 51)
    recon_asd_mu     | (661, 51, 51)
   recon_twin_mu     | (661, 51, 51)
  recon_td_samples   | (100, 841, 51, 51)
 recon_asd_samples   | (100, 661, 51, 51)
 recon_twin_samples  | (100, 661, 51, 51)
CPU times: user 24.4 s, sys: 1.45 s, total: 25.9 s
Wall time: 37.9 s


In [10]:
def loso_crossval(X,Y,model):
    n = X.shape[0]
    Y_hat = []
    for i in tqdm(range(n)):
        svec = np.arange(n)

        train_idx = svec!=i
        test_idx = i

        X_train = X[train_idx,:]
        Y_train = Y[train_idx]

        X_test = X[test_idx,:][np.newaxis,:]
        Y_test = Y[test_idx]

        Y_hat.append(model.fit(X_train, Y_train).predict(X_test))
    Y_hat = np.array(Y_hat)[:,0]
    return Y_hat

In [11]:
key = 'fiq'
vec = df_td[key].values
#vec = (vec-np.nanmin(vec)) / (np.nanmax(vec)-np.nanmin(vec))
e = np.isnan(vec)

X = cmats_td_flat[~e,:]
#X = data['S_mu'][controls,:][~e,:]
Y = vec[~e]

print((X.shape,Y.shape))

k = 100
X_train = X[0:-k,:]
Y_train = Y[0:-k]

X_test = X[-k::,:]
Y_test = Y[-k::]

print((X_train.shape,Y_train.shape))
print((X_test.shape,Y_test.shape))

((793, 1275), (793,))
((693, 1275), (693,))
((100, 1275), (100,))


In [12]:
from sklearn.linear_model import LogisticRegression,LinearRegression,ElasticNet,Ridge,BayesianRidge
from sklearn.neighbors import KNeighborsRegressor 
from sklearn import tree ##tree.DecisionTreeRegressor()
from sklearn.svm import SVR 
from sklearn.ensemble import VotingRegressor
from sklearn.neural_network import MLPRegressor

In [13]:
models = [LinearRegression(),
ElasticNet(l1_ratio=0.5),
ElasticNet(l1_ratio=0.1),
ElasticNet(l1_ratio=0.9),
BayesianRidge(alpha_1=1e-06,alpha_2=1e-06,lambda_1=1e-06,lambda_2=1e-06),
BayesianRidge(alpha_1=1e-03,alpha_2=1e-03,lambda_1=1e-03,lambda_2=1e-03),
BayesianRidge(alpha_1=1e-01,alpha_2=1e-01,lambda_1=1e-01,lambda_2=1e-01),
BayesianRidge(alpha_1=1,alpha_2=1,lambda_1=1,lambda_2=1),
BayesianRidge(alpha_1=2,alpha_2=2,lambda_1=2,lambda_2=2),
BayesianRidge(alpha_1=10,alpha_2=10,lambda_1=10,lambda_2=10),
BayesianRidge(alpha_1=100,alpha_2=100,lambda_1=100,lambda_2=100),
BayesianRidge(alpha_1=250,alpha_2=250,lambda_1=250,lambda_2=250),
KNeighborsRegressor(),
KNeighborsRegressor(n_neighbors=15),
tree.DecisionTreeRegressor(),
SVR(kernel='rbf',degree=3,gamma='scale',coef0=0.0,tol=0.001,C=1.0,epsilon=0.1),
SVR(kernel='rbf',degree=3,gamma='scale',coef0=0.0,tol=0.001,C=100,epsilon=1)]

In [14]:
#r2s = [r2_score(Y_test,model.fit(X_train,Y_train).predict(X_test)) for model in tqdm(models)]

In [15]:
r2s = [r2_score(Y,loso_crossval(X,Y,model)) for model in models]

 19%|█▉        | 154/793 [29:52<2:03:59, 11.64s/it]


KeyboardInterrupt: 

In [ ]:
#r2s = [r2_score(Y_test,model.fit(X_train,Y_train).predict(X_test)) for model in tqdm(models)]

In [ ]:
plt.figure(figsize=(5,15))
n = np.arange(len(r2s))
lbls = [str(model) for model in models]
r2s_plot = r2s.copy()
r2s_plot = np.array(r2s_plot);
r2s_plot[r2s_plot<0]=0;
plt.plot(r2s_plot[-1::-1],n,'*');
plt.yticks(n,labels=lbls[-1::-1],rotation=0);

In [ ]:
# r2_score(Y,loso_crossval(X,Y,BayesianRidge())).round(3)*100

In [ ]:
r2s_plot.max().round(3)*100

In [ ]:
!date